In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from utils import *

gen = generate_mdpp_data()

In [15]:
total_num_passed = 0
total_num_test = 0
total_num_error = 0

case_err = list()

In [ ]:
for test_set in gen:
    task, code_ground_truth, test_list = test_set

    function_name = extract_first_function_name(code_ground_truth)
    prompt_with_name = f"다음 문제를 푸는 파이썬 함수를 작성하세요. test_case를 통과하도록 함수를 작성하세요. 함수 이름은 반드시 `{function_name}` 이어야 합니다.\n\n task: {task} \n test_case: {test_list[0]}\n"

    result = await create_source_code(prompt_with_name)

    code = getattr(result['private'], "executed_code", None)

    num_passed, num_test, num_error = run_tests_exec(code, test_list)

    total_num_passed += num_passed
    total_num_test += num_test
    total_num_error += num_error

    if num_error > 0:
        case_err.append([task, code_ground_truth, test_list])

    print(f"accuracy : {total_num_passed/total_num_test * 100:.2f} passed : {total_num_passed}, num test : {total_num_test}, num err : {total_num_error}")


accuracy : 100.00 passed : 3, num test : 3, num err : 0
[FAIL] Test 2: Assertion failed
accuracy : 83.33 passed : 5, num test : 6, num err : 0
accuracy : 88.89 passed : 8, num test : 9, num err : 0
accuracy : 91.67 passed : 11, num test : 12, num err : 0
accuracy : 93.33 passed : 14, num test : 15, num err : 0
